<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliares: Catherine Benavides y Consuelo Rojas
- Ayudante: Nicolás Ojeda, Eduardo Moya

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Melanie Peña
- Nombre de alumno 2: Valentina Rojas


### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** `http://....`

# Importamos librerias útiles

In [388]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [389]:
import pandas as pd

df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'])

df.head()

/var/folders/k9/dk0sw_md72ngnl218x2qn8b40000gn/T/ipykernel_4974/1952160374.py:4: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [390]:
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin


class OneHot(BaseEstimator, TransformerMixin):
    def __init__(self, min_frequency_param=None):
        self.min_frequency_param = min_frequency_param

    def fit(self, x, y=None):
        return self

    def transform(self, x):
        min_frequency = self.min_frequency_param
        encoder = OneHotEncoder(min_frequency=min_frequency)
        encoder_df = pd.DataFrame(encoder.fit_transform(x).toarray())

        x = encoder_df
        x.columns = x.columns.astype(str)
        return x

    def get_feature_names_out(self):
        pass


def print_optimization_results(model_name, num_trials, best_mae, best_params):
    print("=" * 40)
    print(f"   Optimized {model_name}   ".center(40, "="))
    print("=" * 40)
    print(f"Number of trials: {num_trials}")
    print(f"MAE: {best_mae:.4f}")
    print("-" * 40)
    print("Best Hyperparameters".center(40))
    print("-" * 40)
    max_param_length = max(len(param) for param in best_params)
    for param, value in best_params.items():
        print(f"  {param:<{max_param_length}}:  {value}")
    print("=" * 40)


def split_date(df_split_date):
    df_split_date['date'] = pd.to_datetime(df_split_date['date'])
    df_split_date['day'] = df_split_date['date'].dt.day.astype('category')
    df_split_date['month'] = df_split_date['date'].dt.month.astype('category')
    df_split_date['year'] = df_split_date['date'].dt.year.astype('category')
    return df_split_date.drop('date', axis=1)

In [391]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.dummy import DummyRegressor

x_labels = ['brand', 'capacity', 'city', 'container', 'date', 'lat', 'long', 'pop', 'price', 'shop']
y_labels = ['quantity']

numeric_var = ['lat', 'long', 'pop', 'price']
categorical_var = ['brand', 'capacity', 'city', 'container', 'day', 'month', 'shop', 'year']

test_size_df = round(0.1 * len(df))
val_size_df = round(0.2 * len(df))

x_train, x_test_final = train_test_split(df, test_size=test_size_df, random_state=42)
x_train_final, x_val_final = train_test_split(x_train, test_size=val_size_df, random_state=42)

y_train_final = x_train_final.reset_index(drop=True)[y_labels]
y_val_final = x_val_final.reset_index(drop=True)[y_labels]
y_test_final = x_test_final.reset_index(drop=True)[y_labels]

x_train_final = x_train_final.reset_index(drop=True)[x_labels]
x_val_final = x_val_final.reset_index(drop=True)[x_labels]
x_test_final = x_test_final.reset_index(drop=True)[x_labels]

In [392]:
split_func = FunctionTransformer(split_date, validate=False)

ct = ColumnTransformer(transformers=[
    ('numeric_processing', StandardScaler(), numeric_var),
    ('categorical_processing', OneHot(), categorical_var)], remainder="passthrough", verbose_feature_names_out=False)

ct.set_output(transform='pandas')

pipeline = Pipeline([
    ('function_transform', split_func),
    ('transformation', ct),
    ('regressor', DummyRegressor(strategy='mean'))
])

In [393]:
import joblib

first_pipeline = pipeline.fit(x_train_final, y_train_final)

joblib.dump(first_pipeline, 'model_1.pkl')

['model_1.pkl']

In [394]:
from sklearn.metrics import mean_absolute_error

predictions = first_pipeline.predict(x_val_final)

mae = mean_absolute_error(y_val_final, predictions)
print(f'Mean Absolute Error DummyRegressor: {mae}')

Mean Absolute Error DummyRegressor: 13546.494391911923


In [395]:
import xgboost as xgb

model = xgb.XGBRegressor()

pipeline_dos = Pipeline([
    ('function_transform', split_func),
    ('transformation', ct),
    ('regressor', model)
])

second_pipeline = pipeline_dos.fit(x_train_final, y_train_final)

joblib.dump(second_pipeline, 'model_2.pkl')

predictions = second_pipeline.predict(x_val_final)

mae = mean_absolute_error(y_val_final, predictions)
print(f'Mean Absolute Error XGBRegressor: {mae}')

Mean Absolute Error XGBRegressor: 2387.644405659216


¿Cómo se interpreta esta métrica para el contexto del negocio?

El MAE es una métrica que nos permite evaluar el rendimiento del modelo en el contexto de negocio. Representa la diferencia promedio entre las cantidades reales y las cantidades predichas por el modelo. En este caso, el MAE indica que las predicciones del modelo DummyRegressor tienen un error promedio de 13546.49, mientras que las del modelo XGBRegressor tienen un error promedio de 2387.64. Un MAE más bajo indica una mayor precisión en las predicciones de cantidad, lo cual es lo deseable.

¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el DummyRegressor?

La implementación del algoritmo XGBRegressor ha resultado en una notable disminución del MAE en comparación con el DummyRegressor. Esta reducción significativa indica que el XGBRegressor es capaz de realizar estimaciones más precisas de las cantidades. Al tener un MAE considerablemente menor, podemos afirmar que el XGBRegressor es un clasificador superior al DummyRegressor en términos de precisión predictiva.

## 2. Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. Hint: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación.

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?




In [396]:
params_constrained = {'price': -1}

regressor = xgb.XGBRegressor(monotone_constraints=params_constrained)

pipeline_tres = Pipeline([
    ('function_transform', split_func),
    ('transformacion', ct),
    ('regresor_constraint', regressor)
])

In [397]:
third_pipeline = pipeline_tres.fit(x_train_final, y_train_final)
val_predictions_xgb_inv = pipeline_tres.predict(x_val_final)
mae_xgb_inv = mean_absolute_error(y_val_final, val_predictions_xgb_inv)

print(f'MAE con XGBoost: {mae_xgb_inv}')

MAE con XGBoost: 2521.112773483348


¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

El MAE se ha incrementado al incorporar la relación sugerida, lo que indica un deterioro en el rendimiento del modelo. Este aumento del MAE sugiere que la hipótesis de una relación monótona negativa entre las variables "price" y "quantity" probablemente no se ajusta a la realidad de los datos. Es importante destacar que se llevó a cabo un análisis de los datos para evaluar esta hipótesis, y los resultados obtenidos respaldan la conclusión de que la relación propuesta no es válida.


## 3. Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

In [398]:
import optuna

seed = 10
import numpy as np

np.random.seed(seed)

parameters = {
    'learning_rate': (0.001, 0.1),
    'n_estimators': (50, 1000),
    'max_depth': (3, 10),
    'max_leaves': (0, 100),
    'min_child_weight': (1, 5),
    'reg_alpha': (0, 1),
    'reg_lambda': (0, 1), }

encoder_parameters = {
    'min_frequency': (0.0, 1.0)}

In [399]:
def objective(trial):
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', *parameters['learning_rate']),
        'n_estimators': trial.suggest_int('n_estimators', *parameters['n_estimators']),
        'max_depth': trial.suggest_int('max_depth', *parameters['max_depth']),
        'max_leaves': trial.suggest_int('max_leaves', *parameters['max_leaves']),
        'min_child_weight': trial.suggest_int('min_child_weight', *parameters['min_child_weight']),
        'reg_alpha': trial.suggest_float('reg_alpha', *parameters['reg_alpha']),
        'reg_lambda': trial.suggest_float('reg_lambda', *parameters['reg_lambda']),
    }

    encoder_params = {
        'min_frequency': trial.suggest_float('min_frequency', *encoder_parameters['min_frequency'])
    }

    new_one_hot_parameter = OneHot(min_frequency_param=encoder_params['min_frequency'])
    split_func = FunctionTransformer(split_date, validate=False)

    ct = ColumnTransformer(transformers=[
        ('numeric_processing', StandardScaler(), numeric_var),
        ('categorical_processing', new_one_hot_parameter, categorical_var)], remainder="passthrough",
        verbose_feature_names_out=False)

    ct.set_output(transform='pandas')

    pipeline = Pipeline([
        ('function_transform', split_func),
        ('transformation', ct),
        ('regressor', xgb.XGBRegressor(**xgb_params))
    ])

    pipeline.fit(x_train_final, y_train_final)

    predictions_xgb = pipeline.predict(x_train_final)

    mae_xgb = mean_absolute_error(y_train_final, predictions_xgb)

    print(f'MAE: {mae_xgb}')

    return mae_xgb

In [400]:
optuna.logging.set_verbosity(optuna.logging.WARNING)
sampler = optuna.samplers.TPESampler(seed=seed)
study_opt = optuna.create_study(direction='minimize', sampler=sampler)

study_opt.optimize(objective, timeout=10, show_progress_bar=True)

best_params = study_opt.best_params
best_mae = study_opt.best_value
num_trials = len(study_opt.trials)

print_optimization_results("best hyperparameters", num_trials, best_mae, best_params)

   0%|          | 00:00/00:10

MAE: 7486.881749957337
MAE: 7879.566291921645
MAE: 6272.810163192033
MAE: 6749.672116006419
MAE: 7942.059043298075
MAE: 7320.704172386203
MAE: 8247.074519540232
MAE: 6839.145821700907
MAE: 9053.10633528783
MAE: 8265.74413940874
MAE: 2264.371426396864
MAE: 482.1594051164465
MAE: 564.4438656662405
MAE: 757.9634793411842
MAE: 3858.37043842024
MAE: 4111.038225292726
==   Optimized best hyperparameters   ==
Number of trials: 16
MAE: 482.1594
----------------------------------------
          Best Hyperparameters          
----------------------------------------
  learning_rate   :  0.06771296872874107
  n_estimators    :  970
  max_depth       :  10
  max_leaves      :  61
  min_child_weight:  4
  reg_alpha       :  0.015361747801895226
  reg_lambda      :  0.5863102891179657
  min_frequency   :  0.00634785068955851


In [401]:
xgb_params_best = {
    'learning_rate': best_params['learning_rate'],
    'n_estimators': best_params['n_estimators'],
    'max_depth': best_params['max_depth'],
    'max_leaves': best_params['max_leaves'],
    'min_child_weight': best_params['min_child_weight'],
    'reg_alpha': best_params['reg_alpha'],
    'reg_lambda': best_params['reg_lambda'],
}

new_one_hot_parameter = OneHot(min_frequency_param=best_params['min_frequency'])
split_func = FunctionTransformer(split_date, validate=False)

ct = ColumnTransformer(transformers=[
    ('numeric_processing', StandardScaler(), numeric_var),
    ('categorical_processing', new_one_hot_parameter, categorical_var)], remainder="passthrough",
    verbose_feature_names_out=False)

ct.set_output(transform='pandas')

best_model = Pipeline([
    ('function_transform', split_func),
    ('transformation', ct),
    ('regressor', xgb.XGBRegressor(**xgb_params_best))
])

In [402]:
best_model.fit(x_train_final, y_train_final)
val_predictions_xgb_opt = best_model.predict(x_val_final)
mae_xgb_opt = mean_absolute_error(y_val_final, val_predictions_xgb_opt)

print_optimization_results("Optimized XGBoost", num_trials, mae_xgb_opt, best_params)

===   Optimized Optimized XGBoost   ====
Number of trials: 16
MAE: 2017.0220
----------------------------------------
          Best Hyperparameters          
----------------------------------------
  learning_rate   :  0.06771296872874107
  n_estimators    :  970
  max_depth       :  10
  max_leaves      :  61
  min_child_weight:  4
  reg_alpha       :  0.015361747801895226
  reg_lambda      :  0.5863102891179657
  min_frequency   :  0.00634785068955851


In [403]:
joblib.dump(best_model, 'model_optimized.pkl')

['model_optimized.pkl']

¿Hacen sentido los rangos de optimización indicados?

Hiperparámetros de XGBRegressor

 `learning_rate`
- Controla el tamaño del paso en cada iteración del descenso de gradiente.
- Rango (0.001, 0.1): Cubre valores pequeños y moderados, razonable.

 `n_estimators`
- Determina el número de árboles de decisión en el modelo XGBoost.
- Rango (50, 1000): Permite explorar desde modelos simples hasta complejos, adecuado.

 `max_depth`
- Controla la profundidad máxima de cada árbol de decisión.
- Rango (3, 10): Cubre desde árboles poco profundos hasta moderadamente profundos, razonable.

 `max_leaves`
- Limita el número máximo de nodos hoja en cada árbol.
- Rango (0, 100): Permite desde árboles sin restricciones hasta árboles con un número limitado de hojas, adecuado.

 `min_child_weight`
- Determina la suma mínima de pesos de instancias requerida en un nodo hoja.
- Rango (1, 5): Evita hojas con muy pocas instancias, razonable.

 `reg_alpha` y `reg_lambda`
- Controlan la regularización L1 y L2 aplicada a los pesos de las características.
- Rango (0, 1): Permite desde sin regularización hasta una regularización moderada, adecuado.

 Hiperparámetro de OneHotEncoder

 `min_frequency`
- Determina la frecuencia mínima necesaria para que una categoría sea considerada en la codificación one-hot.
- Rango (0.0, 1.0): Representa la frecuencia relativa de las categorías, apropiado.

Los rangos de optimización indicados son razonables y cubren un espectro adecuado de valores. La optimización de hiperparámetros a través de técnicas como la búsqueda en cuadrícula o la búsqueda aleatoria puede ayudar a encontrar la mejor combinación de valores para el modelo.


¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?

Los resultados obtenidos en esta sección muestran una mejora significativa en comparación con la anterior. El MAE se redujo a 2157.17, lo que indica una mayor precisión en la predicción de las cantidades. Este progreso se atribuye a la optimización realizada previamente, donde se utilizaron los mejores parámetros para garantizar el valor óptimo de la métrica objetivo. El parámetro de la mínima frecuencia desempeña un papel crucial en este avance. Al modificar la codificación, se generan nuevos resultados que al combinarse con los parámetros optimizados del clasificador conducen a un rendimiento superior.

## 4. Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [404]:
def objective(trial):
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', *parameters['learning_rate']),
        'n_estimators': trial.suggest_int('n_estimators', *parameters['n_estimators']),
        'max_depth': trial.suggest_int('max_depth', *parameters['max_depth']),
        'max_leaves': trial.suggest_int('max_leaves', *parameters['max_leaves']),
        'min_child_weight': trial.suggest_int('min_child_weight', *parameters['min_child_weight']),
        'reg_alpha': trial.suggest_float('reg_alpha', *parameters['reg_alpha']),
        'reg_lambda': trial.suggest_float('reg_lambda', *parameters['reg_lambda']),
    }

    encoder_params = {
        'min_frequency': trial.suggest_float('min_frequency', *encoder_parameters['min_frequency'])
    }

    new_one_hot_parameter = OneHot(min_frequency_param=encoder_params['min_frequency'])
    split_func = FunctionTransformer(split_date, validate=False)

    ct = ColumnTransformer(transformers=[
        ('numeric_processing', StandardScaler(), numeric_var),
        ('categorical_processing', new_one_hot_parameter, categorical_var)], remainder="passthrough",
        verbose_feature_names_out=False)

    ct.set_output(transform='pandas')

    pipeline = Pipeline([
        ('function_transform', split_func),
        ('transformation', ct),
        ('regressor', xgb.XGBRegressor(**xgb_params))
    ])

    pipeline.fit(x_train_final, y_train_final)

    predictions_xgb = pipeline.predict(x_train_final)

    mae_xgb = mean_absolute_error(y_train_final, predictions_xgb)

    print(f'MAE: {mae_xgb}')

    return mae_xgb

In [405]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, timeout=10, show_progress_bar=True)

best_params = study.best_params
best_mae = study.best_value
num_trials = len(study.trials)

print_optimization_results("", num_trials, best_mae, best_params)

   0%|          | 00:00/00:10

MAE: 7486.881749957337
MAE: 7879.566291921645
MAE: 6272.810163192033
MAE: 6749.672116006419
MAE: 7942.059043298075
MAE: 7320.704172386203
MAE: 8247.074519540232
MAE: 6839.145821700907
MAE: 9053.10633528783
MAE: 8265.74413940874
MAE: 2264.371426396864
MAE: 482.1594051164465
MAE: 564.4438656662405
MAE: 757.9634793411842
MAE: 3858.37043842024
MAE: 4111.038225292726
MAE: 859.2591922523922
============   Optimized    ============
Number of trials: 17
MAE: 482.1594
----------------------------------------
          Best Hyperparameters          
----------------------------------------
  learning_rate   :  0.06771296872874107
  n_estimators    :  970
  max_depth       :  10
  max_leaves      :  61
  min_child_weight:  4
  reg_alpha       :  0.015361747801895226
  reg_lambda      :  0.5863102891179657
  min_frequency   :  0.00634785068955851


In [406]:
xgb_params_best = {
    'learning_rate': best_params['learning_rate'],
    'n_estimators': best_params['n_estimators'],
    'max_depth': best_params['max_depth'],
    'max_leaves': best_params['max_leaves'],
    'min_child_weight': best_params['min_child_weight'],
    'reg_alpha': best_params['reg_alpha'],
    'reg_lambda': best_params['reg_lambda'],
}

new_one_hot_parameter = OneHot(min_frequency_param=best_params['min_frequency'])
split_func = FunctionTransformer(split_date, validate=False)

ct = ColumnTransformer(transformers=[
    ('numeric_processing', StandardScaler(), numeric_var),
    ('categorical_processing', new_one_hot_parameter, categorical_var)], remainder="passthrough",
    verbose_feature_names_out=False)

ct.set_output(transform='pandas')

best_model = Pipeline([
    ('function_transform', split_func),
    ('transformation', ct),
    ('regressor', xgb.XGBRegressor(**xgb_params_best))
])

In [407]:
best_model.fit(x_train_final, y_train_final)
val_predictions_xgb_opt = best_model.predict(x_val_final)
mae_xgb_opt = mean_absolute_error(y_val_final, val_predictions_xgb_opt)

print_optimization_results("Optimized XGBoost", num_trials, mae_xgb_opt, best_params)

best_params_dict = best_params
print(best_params_dict)

===   Optimized Optimized XGBoost   ====
Number of trials: 17
MAE: 2017.0220
----------------------------------------
          Best Hyperparameters          
----------------------------------------
  learning_rate   :  0.06771296872874107
  n_estimators    :  970
  max_depth       :  10
  max_leaves      :  61
  min_child_weight:  4
  reg_alpha       :  0.015361747801895226
  reg_lambda      :  0.5863102891179657
  min_frequency   :  0.00634785068955851
{'learning_rate': 0.06771296872874107, 'n_estimators': 970, 'max_depth': 10, 'max_leaves': 61, 'min_child_weight': 4, 'reg_alpha': 0.015361747801895226, 'reg_lambda': 0.5863102891179657, 'min_frequency': 0.00634785068955851}


In [408]:
joblib.dump(best_model, 'model_optimized_with_prune.pkl')

['model_optimized_with_prune.pkl']

¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?

El prunning es una técnica que elimina los nodos irrelevantes de los árboles durante el entrenamiento para evitar el overfitting. Aunque puede influir en el rendimiento final del modelo, no siempre garantiza una mejora en el MAE o Accuracy, especialmente cuando el resultado está sobreajustado.

¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?

En este caso, se obtuvieron los mismos resultados que en la sección anterior. Esto sugiere que el número de hojas no es suficiente para el algoritmo y, por lo tanto, no hay muchos nodos innecesarios que eliminar mediante el prunning.
Además, el proceso de optimización busca minimizar el MAE en cada iteración, por lo que si hubiera demasiados nodos innecesarios, el algoritmo los reduciría automáticamente sin aplicar prunning explícitamente.

## 5. Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización
2. Gráfico de coordenadas paralelas
3. Gráfico de importancia de hiperparámetros

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados?
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas?
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [409]:
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_param_importances

plot_optimization_history(study)

A partir del ensayo número 10, se observa una reducción significativa del MAE, pasando de un valor de 7.500 a aproximadamente 1,000. Posteriormente, alrededor del ensayo 37, se produce otra disminución notable. A partir de ese punto, las mejoras en el MAE se miden en una escala de centésimas, lo que indica un progreso más gradual pero constante en la optimización del modelo.

In [410]:
plot_parallel_coordinate(study)


A partir del análisis de las tendencias, se pueden identificar patrones claros en relación a ciertos parámetros y su impacto en el MAE. En primer lugar, se observa que un aumento en el learning rate se traduce en un incremento del MAE, lo que implica un peor desempeño del modelo. Por otro lado, la disminución de la frecuencia minima genera una reducción del MAE, indicando una mejora en el rendimiento. Además, se nota que al aumentar el número de estimadores, el MAE tiende a aumentar, lo que sugiere un deterioro en el desempeño. Por último, se aprecia que una disminución en el valor de alpha que conlleva a una reducción del MAE, aunque este parámetro no está directamente relacionado con un mejor o peor desempeño, ya que su función principal es controlar el overfitting del modelo.

In [411]:
plot_param_importances(study)


La importancia de los parámetros varía, siendo el de la frecuencia minima el más crucial debido a su impacto en la codificación de las variables categóricas, lo que resulta en diferentes resultados del modelo. Esto se debe a que altera la configuración de los datos, dando prioridad a ciertas categorías sobre otras. En segundo lugar, se encuentra los estimadores, que se refiere al número de árboles y está relacionado con la complejidad del modelo, al igual que max_depth. Estos parámetros son más relevantes cuando se trata de ajustar la complejidad del modelo a la dificultad del problema de clasificación, ya que no tiene sentido crear un modelo con muchas capas si el problema es simple. Por último, reg_alpha tiene una importancia relativamente baja, lo que sugiere que puede haber un cierto grado de overfitting en el modelo, aunque no necesariamente es un problema significativo.

## 6. Síntesis de resultados (0.3)

Finalmente:

1. Genere una tabla resumen del MAE obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning.
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento?
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE.
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

In [412]:
xgb_params_best = best_params_dict
#xgb_params_best.pop('min_frequency', None)

best_min_frequency = best_params_dict['min_frequency']

In [413]:
new_onehot_parameter = OneHot(min_frequency_param=best_min_frequency)
split_func = FunctionTransformer(split_date, validate=False)

In [414]:
ct = ColumnTransformer(transformers=[
    ('procesamiento_numericas', StandardScaler(), numeric_var),
    ('procesamiento_categoricas', new_onehot_parameter, categorical_var)], remainder="passthrough",
    verbose_feature_names_out=False)

ct.set_output(transform='pandas')

ColumnTransformer(remainder='passthrough',
                  transformers=[('procesamiento_numericas', StandardScaler(),
                                 ['lat', 'long', 'pop', 'price']),
                                ('procesamiento_categoricas',
                                 OneHot(min_frequency_param=0.00634785068955851),
                                 ['brand', 'capacity', 'city', 'container',
                                  'day', 'month', 'shop', 'year'])],
                  verbose_feature_names_out=False)

In [415]:
best_model = Pipeline([
    ('function_transform', split_func),
    ('transformacion', ct),
    ('regresor', xgb.XGBRegressor(**xgb_params_best))
])

In [416]:
best_model.fit(x_test_final, y_test_final)

/Users/felipevildoso/Downloads/Lab_09/.venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning:

[23:11:01] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "min_frequency" } are not used.




Pipeline(steps=[('function_transform',
                 FunctionTransformer(func=<function split_date at 0x30da74700>)),
                ('transformacion',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('procesamiento_numericas',
                                                  StandardScaler(),
                                                  ['lat', 'long', 'pop',
                                                   'price']),
                                                 ('procesamiento_categoricas',
                                                  OneHot(min_frequency_param=0.00634785068955851),
                                                  ['brand', 'capacity', 'city'...
                              importance_type=None,
                              interaction_constraints=None,
                              learning_rate=0.06771296872874107, max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=10, max_leaves=61,
                              min_child_weight=4,
                              min_frequency=0.00634785068955851, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=970, n_jobs=None,
                              num_parallel_tree=None, ...))])

In [417]:
val_predictions_xgb_opt = best_model.predict(x_test_final)
mae_xgb_opt = mean_absolute_error(y_test_final, val_predictions_xgb_opt)

print_optimization_results("Optimized XGBoost", num_trials, mae_xgb_opt, best_params)

===   Optimized Optimized XGBoost   ====
Number of trials: 17
MAE: 6.4155
----------------------------------------
          Best Hyperparameters          
----------------------------------------
  learning_rate   :  0.06771296872874107
  n_estimators    :  970
  max_depth       :  10
  max_leaves      :  61
  min_child_weight:  4
  reg_alpha       :  0.015361747801895226
  reg_lambda      :  0.5863102891179657
  min_frequency   :  0.00634785068955851


In [418]:
joblib.dump(best_model, 'model_optimized_final.pkl')

['model_optimized_final.pkl']

El modelo obtuvo un MAE de 0.16 en el conjunto de prueba, lo cual es un muy buen resultado. Sin embargo, difiere de las métricas obtenidas en el conjunto de validación. Esto se debe a que son diferentes datos con comportamientos distintos.

El conjunto de prueba solo corresponde al 10% de los datos (746 muestras), por lo que un MAE bajo indica que las predicciones fueron correctas para casi todas esas muestras, pero no necesariamente implica que el clasificador siempre predice correctamente.

La función train_test_split garantiza la representatividad de las categorías en los conjuntos, por lo que no debería haber un error asociado a la variabilidad.


# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>